In [ ]:
import os
#os.chdir("/home/heuzerothp/wikiwho_tsne")

import pandas as pd
from wikiwho_chobj import Chobjer

import sys
sys.path.append('../utils/')
from _vars import *



import nltk
nltk.download("stopwords")
LEFT_CONTEXT = 4
RIGHT_CONTEXT = 4


co = Chobjer(article="39570", pickles_path= '../../bert', lang='en', context=max(LEFT_CONTEXT, RIGHT_CONTEXT))


df = pd.DataFrame(co.iter_chobjs(), columns = next(co.iter_chobjs()).keys())
df=df[(df['ins_tokens_str'].str.len() + df['del_tokens_str'].str.len()) <= 20]


jlogie = pd.read_csv("../../John_Logie_Baird_FULL.csv")






In [ ]:
import numpy as np

from merge import combine

merged = df.apply(lambda x: combine(x, jlogie), axis=1)

# captures if we also want to use changeobjects that do not have tokens that are ground-truth labelled
OUTER_JOIN = True

merged = merged.dropna(how="all")
if not OUTER_JOIN:
    merged = merged[(merged["birth_place"].isna() & merged["Bulk"].isna() & merged["nationality"].isna() & merged["Link"].isna())== False]

In [ ]:
print(merged.loc[92,'right_token_str'],'\n', merged.loc[93,'right_token_str'],  '\n',merged.loc[92,'left_token_str'],'\n', merged.loc[93,'left_token_str'])

In [ ]:
print(merged.loc[92,'right_token'],'\n', merged.loc[93,'right_token'],  '\n',merged.loc[92,'left_token'],'\n', merged.loc[93,'left_token'])

### remove Bulks

In [ ]:
# remove gaps longer than 20 tokens
if GAP:
    
    
    merged = merged[(merged['ins_tokens_str'].str.len() + merged['del_tokens_str'].str.len()) <= 20]

In [ ]:
import re
def making_list(tokens):
    new_tokens = []
    for item in tokens:
        new_tokens.append(' '.join(word for word in item if re.match("^[a-zA-Z0-9_]*$", word)))

    return(new_tokens)


all_tokens = []
merged = merged.reset_index()
#merged=merged.drop(['level_0'], axis=1)
for i, row in merged.iterrows():
    all_tokens.append(making_list([row['left_token_str'],row['right_token_str']]))#,row['ins_tokens_str'],row['del_tokens_str']]))
    merged.loc[i, ['left_token_str','right_token_str']] = all_tokens[i]
    #merged.loc[i,'right_token_str'] = all_tokens[i][1]
    merged.loc[i,'ins_tokens_str'] = making_list([row['ins_tokens_str']])
    merged.loc[i,'del_tokens_str'] = making_list([row['del_tokens_str']])

In [ ]:
merged

##  Embed words by creating a vector of length 300 for each inserted and deleted tokens, so the resulting vector for one change object has length 600

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

embed = load_vectors('../../wiki-news-300d-1M-subword.vec')

In [ ]:
from nltk.corpus import stopwords
import numpy as np
from gensim.sklearn_api import W2VTransformer
from gensim.models import KeyedVectors
from copy import deepcopy
import pdb

def transform(phrase, embedding):
    li_vecs = []
    for i in range(len(phrase)):
        if phrase[i] in embedding:
            li_vecs.append(list(deepcopy(embedding[phrase[i]])))
    if len(li_vecs) != 0:
        vecs = np.stack(li_vecs)
        
        return vecs            
    else:
        return None

def filter_stopwords(phrase):
    important_words = []
    for word in phrase.split():
        if word not in stopwords.words('english'):
            important_words.append(word)
    return important_words

def create_features(chobj, use_gap, left_context, right_context):
    if left_context > 0:
        left_wordvecs = transform(filter_stopwords(chobj["left_token_str"]), embed)
        if left_wordvecs is None:
            left_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            left_wordvecs = np.mean(left_wordvecs, axis=0)
    if right_context > 0:
        right_wordvecs = transform(filter_stopwords(chobj["right_token_str"]), embed)  
        if right_wordvecs is None:
            right_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            right_wordvecs = np.mean(right_wordvecs, axis=0)
    if use_gap:
        ins_wordvecs = transform(filter_stopwords(chobj["ins_tokens_str"]), embed)     
        del_wordvecs = transform(filter_stopwords(chobj["del_tokens_str"]), embed)
        if ins_wordvecs is None:
            ins_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            ins_wordvecs = np.mean(ins_wordvecs, axis=0)
        if del_wordvecs is None:
            del_wordvecs = np.full(WORD_EMBED_SIZE, 0)
        else:
            del_wordvecs = np.mean(del_wordvecs, axis=0)
    
    li = []
    for a in ["left_wordvecs", "right_wordvecs", "ins_wordvecs", "del_wordvecs"]:
        if a in vars():
            li.append(vars()[a])
    
    try:
        feat = pd.Series(np.nan_to_num(np.concatenate(li)))
    except ValueError:
        pdb.set_trace()
    
    
    return feat

Embedded = merged.apply(lambda x: create_features(x, use_gap=GAP, left_context=LEFT_CONTEXT, right_context=RIGHT_CONTEXT), 
                        axis=1)

## (a) K-means clustering


In [ ]:
from sklearn.cluster import KMeans

clusterer = KMeans(random_state=42)
clusters = clusterer.fit_predict(Embedded)

In [ ]:
pd.Series(clusters).unique()


### find closest words to centroids for labeling clusters

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def average_vectors_to_shape_300(vec):
    result = np.zeros(300)
    fold = int(vec.shape[0]/300)
    for i in range(300):
        to_avg = []
        for k in range(fold):
            to_avg.append(vec[i + k *300])
        result[i] = np.mean(to_avg)
    return result

centroids = clusterer.cluster_centers_
centroids_300 = [np.zeros(300) for i in range(len(centroids))]
for i in range(len(centroids)):
    centroids_300[i] = average_vectors_to_shape_300(centroids[i])

embed_keys = list(embed.keys())
embed_vals = []
for i in embed.values():
    embed_vals.append(list(deepcopy(i)))
    
X = np.array(embed_vals)
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
dists, inds = nbrs.kneighbors(centroids_300)

closest_words = {}
for i in range(len(pd.Series(clusters).unique())):
    print(i)
    closest_words[i] = [(embed_keys[inds[i, j]], dists[i, j]) for j in range(len(inds[i]))]

## (b) DBSCAN clustering


In [ ]:
from sklearn.cluster import DBSCAN

clusterer = DBSCAN(eps=0.5, min_samples=5)
clusters = clusterer.fit_predict(Embedded)
labels = clusterer.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
pd.Series(clusters).unique()

## t-SNE


In [ ]:
from sklearn.manifold import TSNE

X = TSNE(random_state=42).fit_transform(Embedded)


# Save data for plotting


In [ ]:
# saves data for plotting

if not (merged.index == range(len(merged))).all():
    merged = merged.reset_index()
    
    
plot_data = pd.concat([pd.DataFrame(X), pd.Series(clusters), merged], axis=1)

plot_data.columns = ['t-SNE-X', 't-SNE-Y', 'cluster', 'index','Bulk', 
                     
                  'Link',          'action',     'birth_place',
           'del_end_pos',   'del_start_pos',      'del_tokens',
        'del_tokens_str',          'editor',        'from_rev',
        'from_timestamp',     'ins_end_pos',   'ins_start_pos',
            'ins_tokens',  'ins_tokens_str',      'left_neigh',
            'left_token',  'left_token_str',     'nationality',
               'page_id',     'right_neigh',     'right_token',
       'right_token_str',            'text',          'to_rev',
          'to_timestamp',           'token']    


plot_data.to_csv("../plotData_jlb_outer_dbscan_eps0_5.csv")


